#LAB 1:  Ceaser Cipher

In [ ]:
def caesar_cipher(text, shift):
    result = ""

    for char in text:
        if char.isupper():
            result += chr((ord(char) + shift - 65) % 26 + 65)
        elif char.islower():
            result += chr((ord(char) + shift - 97) % 26 + 97)
        elif char.isdigit():
            result += chr((ord(char) + shift - 48) % 10 + 48)
        else:
            result += char

    return result

def caesar_decrypt(ciphertext, shift):
    return caesar_cipher(ciphertext, -shift)

# Input from user
plaintext = input("Enter the text: ")
shift = int(input("Enter the shift value (integer): "))

# Encrypt the plaintext
ciphertext = caesar_cipher(plaintext, shift)

# Decrypt the ciphertext
decrypted_text = caesar_decrypt(ciphertext, shift)

# Display both results
print("\n=============================== Result ================================")
print(f"Plaintext: {plaintext}")
print(f"Encrypted text: {ciphertext}")
print(f"Decrypted text: {decrypted_text}")
print("======================================================================")


Enter the text: yes
Enter the shift value (integer): 5

=============================== Result ================================
Plaintext: yes
Encrypted text: djx
Decrypted text: yes


# LAB 2: Playfair Cipher


The Playfair cipher is a classical encryption technique that encrypts pairs of letters (bigrams) instead of individual letters. It was invented by Charles Wheatstone in 1854 and later popularized by Sir Frances Bacon. Here’s a guide on how to encrypt and decrypt text using the Playfair cipher.

## Overview

1. **Generate the Cipher Grid:**
   - Create a 5x5 grid of letters based on a keyword. This grid will contain all letters of the alphabet (usually combining I and J into one letter to fit 25 letters into the grid).
   - The grid is constructed by placing the letters of the keyword first and then filling in the remaining letters of the alphabet in order.

2. **Prepare the Plaintext:**
   - Convert the plaintext to uppercase and replace any occurrences of 'J' with 'I'.
   - Split the plaintext into pairs of letters. If a pair is made of identical letters, replace the second letter with 'X'. If there’s an odd number of letters, append 'X' to the last letter.

3. **Encrypt Each Pair:**
   - Locate each letter of the pair in the grid.
   - Apply the Playfair cipher rules based on their positions in the grid:
     - **Same Row:** Replace each letter with the letter immediately to its right (wrap around to the beginning of the row if necessary).
     - **Same Column:** Replace each letter with the letter immediately below it (wrap around to the top of the column if necessary).
     - **Rectangle:** Replace each letter with the letter in its own row but in the column of the other letter of the pair.

4. **Decrypt Each Pair:**
   - Follow the reverse of the encryption rules:
     - **Same Row:** Replace each letter with the letter immediately to its left (wrap around to the end of the row if necessary).
     - **Same Column:** Replace each letter with the letter immediately above it (wrap around to the bottom of the column if necessary).
     - **Rectangle:** Replace each letter with the letter in its own row but in the column of the other letter of the pair.

## Example

### Step 1: Generate the Cipher Grid

Given the keyword "KEYWORD":

1. Create a grid with the keyword letters, removing duplicates:

`K E Y W O R A B C D F G H I/J L M N P Q S T U V X Z`

### Step 2: Prepare the Plaintext

For plaintext "HELLO WORLD":
1. Convert to uppercase and replace 'J' with 'I': "HELLO WORLD"
2. Split into pairs: "HE", "LL", "OW", "OR", "LD" (add 'X' if necessary)

### Step 3: Encrypt Each Pair

Using the grid:
- **HE:** Located in the rectangle formed by (H, E) → 'GB'
- **LL:** Same row, replace with 'XX'
- **OW:** Located in the rectangle formed by (O, W) → 'RO'
- **OR:** Located in the rectangle formed by (O, R) → 'KW'
- **LD:** Located in the rectangle formed by (L, D) → 'DN'

### Step 4: Decrypt Each Pair

Apply the reverse of the encryption rules to recover the original plaintext.

## Playfair Cipher Rules

1. **Same Row:** Letters are replaced by the letter to their immediate right.
2. **Same Column:** Letters are replaced by the letter immediately below.
3. **Rectangle:** Letters are replaced by letters on the same row but in the column of the other letter.

By following these steps and applying the rules, you can successfully encrypt and decrypt messages using the Playfair cipher.


In [ ]:
import numpy as np

def create_key_matrix(key):
    key = ''.join(sorted(set(key), key=lambda x: key.index(x)))
    key = key.replace('J', 'I')  # Combine 'J' and 'I'

    matrix = np.zeros((5, 5), dtype=object)
    alphabet = 'ABCDEFGHIKLMNOPQRSTUVWXYZ'

    used_chars = set()
    i, j = 0, 0

    for char in key:
        if char not in used_chars and char in alphabet:
            matrix[i][j] = char
            used_chars.add(char)
            j += 1
            if j == 5:
                j = 0
                i += 1

    for char in alphabet:
        if char not in used_chars:
            matrix[i][j] = char
            used_chars.add(char)
            j += 1
            if j == 5:
                j = 0
                i += 1

    return matrix

def preprocess_text(text):
    text = text.upper().replace('J', 'I').replace(' ', '')
    processed_text = ''
    i = 0

    while i < len(text):
        if i + 1 < len(text) and text[i] == text[i + 1]:
            processed_text += text[i] + 'X'
            i += 1
        else:
            processed_text += text[i]
            i += 1

    if len(processed_text) % 2 != 0:
        processed_text += 'X'

    return [processed_text[i:i+2] for i in range(0, len(processed_text), 2)]

def encrypt_digraph(digraph, matrix):
    pos1 = np.where(matrix == digraph[0])
    pos2 = np.where(matrix == digraph[1])

    if pos1[0][0] == pos2[0][0]:
        return matrix[pos1[0][0], (pos1[1][0] + 1) % 5] + matrix[pos2[0][0], (pos2[1][0] + 1) % 5]
    elif pos1[1][0] == pos2[1][0]:
        return matrix[(pos1[0][0] + 1) % 5, pos1[1][0]] + matrix[(pos2[0][0] + 1) % 5, pos2[1][0]]
    else:
        return matrix[pos1[0][0], pos2[1][0]] + matrix[pos2[0][0], pos1[1][0]]

def decrypt_digraph(digraph, matrix):
    pos1 = np.where(matrix == digraph[0])
    pos2 = np.where(matrix == digraph[1])

    if pos1[0][0] == pos2[0][0]:
        return matrix[pos1[0][0], (pos1[1][0] - 1) % 5] + matrix[pos2[0][0], (pos2[1][0] - 1) % 5]
    elif pos1[1][0] == pos2[1][0]:
        return matrix[(pos1[0][0] - 1) % 5, pos1[1][0]] + matrix[(pos2[0][0] - 1) % 5, pos2[1][0]]
    else:
        return matrix[pos1[0][0], pos2[1][0]] + matrix[pos2[0][0], pos1[1][0]]

def playfair_encrypt(key, text):
    key_matrix = create_key_matrix(key)
    digraphs = preprocess_text(text)
    ciphertext = ''.join(encrypt_digraph(digraph, key_matrix) for digraph in digraphs)
    return ciphertext

def playfair_decrypt(key, text):
    key_matrix = create_key_matrix(key)
    digraphs = [text[i:i+2] for i in range(0, len(text), 2)]
    plaintext = ''.join(decrypt_digraph(digraph, key_matrix) for digraph in digraphs)
    return plaintext

# Input from user
key = "MONARCHY"
plaintext = input("Enter the plaintext: ").strip()

# Encrypt the plaintext
ciphertext = playfair_encrypt(key, plaintext)

# Decrypt the ciphertext
decrypted_text = playfair_decrypt(key, ciphertext)

# Display both results
print("\n=============================== Result ================================")
print(f"Plaintext: {plaintext}")
print(f"Encrypted text: {ciphertext}")
print(f"Decrypted text: {decrypted_text}")
print("======================================================================")


Enter the plaintext: yes

=============================== Result ================================
Plaintext: yes
Encrypted text: CGXA
Decrypted text: YESX



# LAB 3: Vigenère Cipher

The Vigenère cipher is a method of encrypting alphabetic text using a series of Caesar ciphers based on the letters of a keyword. This method shifts each letter of the plaintext by a number of positions defined by the corresponding letter of the keyword. Here's a brief guide on how to encrypt and decrypt text using the Vigenère cipher.

## Encryption

1. **Prepare the Keyword:**
   - Repeat the keyword to match the length of the plaintext.

2. **Encrypt Each Character:**
   - For each letter in the plaintext:
     - Determine the shift amount using the corresponding letter in the keyword.
     - Apply the shift to the plaintext letter. Convert letters to their alphabetical index (A=0, B=1, ..., Z=25), apply the shift, and convert back to a letter.

3. **Handle Case Sensitivity:**
   - Preserve the case of the letters. If the plaintext letter is uppercase, the result should be uppercase, and similarly for lowercase letters.
   - Non-alphabetic characters remain unchanged.

### Encryption Formula

For each letter `P` in plaintext and corresponding letter `K` in the keyword:

$$ C = (P + K) \mod 26 $$

where:
- `P` is the zero-based index of the plaintext letter.
- `K` is the zero-based index of the keyword letter.
- `C` is the zero-based index of the ciphertext letter.

## Decryption

1. **Prepare the Keyword:**
   - Repeat the keyword to match the length of the ciphertext.

2. **Decrypt Each Character:**
   - For each letter in the ciphertext:
     - Determine the shift amount using the corresponding letter in the keyword.
     - Apply the inverse shift to the ciphertext letter. Convert letters to their alphabetical index, apply the inverse shift, and convert back to a letter.

3. **Handle Case Sensitivity:**
   - Preserve the case of the letters similarly to the encryption process.
   - Non-alphabetic characters remain unchanged.

### Decryption Formula

For each letter `C` in ciphertext and corresponding letter `K` in the keyword:

$$ P = (C - K + 26) \mod 26 $$

where:
- `C` is the zero-based index of the ciphertext letter.
- `K` is the zero-based index of the keyword letter.
- `P` is the zero-based index of the plaintext letter.

## Example

For plaintext "HELLO" and keyword "KEY":

1. **Repeat Keyword:** "KEYKE"
2. **Encryption:**
   - 'H' (7) shifted by 'K' (10) becomes 'R' (17)
   - 'E' (4) shifted by 'E' (4) becomes 'I' (8)
   - Repeat similarly for the rest of the letters.

3. **Decryption:**
   - 'R' (17) shifted back by 'K' (10) becomes 'H' (7)
   - 'I' (8) shifted back by 'E' (4) becomes 'E' (4)
   - Repeat similarly for the rest of the letters.

This process allows you to both encrypt and decrypt messages using the Vigenère cipher by following these steps and formulas.


In [ ]:
def vigenere_encrypt(plaintext, keyword):
    ciphertext = ""
    keyword_length = len(keyword)

    for i in range(len(plaintext)):
        plain_char = plaintext[i]
        key_char = keyword[i % keyword_length].upper()
        shift = ord(key_char) - ord('A')

        if plain_char.isalpha():
            base = ord('a') if plain_char.islower() else ord('A')
            encrypted_char = chr((ord(plain_char) - base + shift) % 26 + base)
            ciphertext += encrypted_char
        else:
            ciphertext += plain_char

    return ciphertext

def vigenere_decrypt(ciphertext, keyword):
    plaintext = ""
    keyword_length = len(keyword)

    for i in range(len(ciphertext)):
        cipher_char = ciphertext[i]
        key_char = keyword[i % keyword_length].upper()
        shift = ord(key_char) - ord('A')

        if cipher_char.isalpha():
            base = ord('a') if cipher_char.islower() else ord('A')
            decrypted_char = chr((ord(cipher_char) - base - shift) % 26 + base)
            plaintext += decrypted_char
        else:
            plaintext += cipher_char

    return plaintext

# Input from user
plaintext = input("Enter the plaintext: ").strip()
keyword = input("Enter the keyword: ").strip()

# Encrypt the plaintext
ciphertext = vigenere_encrypt(plaintext, keyword)

# Decrypt the ciphertext
decrypted_text = vigenere_decrypt(ciphertext, keyword)

# Display both results
print("\n=============================== Result ================================")
print(f"Plaintext: {plaintext}")
print(f"Encrypted text: {ciphertext}")
print(f"Decrypted text: {decrypted_text}")
print("======================================================================")


Enter the plaintext: yes
Enter the keyword: ok

=============================== Result ================================
Plaintext: yes
Encrypted text: mog
Decrypted text: yes


# LAB 4: Hill Cipher

The Hill cipher is a polygraphic substitution cipher based on linear algebra. It encrypts blocks of text using matrix multiplication. Here's a guide on how to encrypt and decrypt text using the Hill cipher.

## Overview

1. **Key Matrix:**
   - Choose a key matrix of size `n x n` where `n` is the block size (typically 2 or 3). The matrix must be invertible modulo 26 (the number of letters in the alphabet).

2. **Prepare the Plaintext:**
   - Convert the plaintext to uppercase and remove non-alphabetic characters.
   - Pad the plaintext if necessary to make its length a multiple of `n`. For example, if using a 2x2 matrix, make sure the plaintext length is a multiple of 2.

3. **Encrypt Each Block:**
   - Divide the plaintext into blocks of size `n`.
   - Convert each block into a column vector of size `n x 1`.
   - Multiply the key matrix by the column vector modulo 26 to get the ciphertext vector.

4. **Decrypt Each Block:**
   - Compute the inverse of the key matrix modulo 26.
   - Multiply the inverse matrix by the ciphertext vector modulo 26 to get the plaintext vector.

## Example

### Key Matrix

Suppose we use a 2x2 matrix for encryption with the key matrix:

\[ K = \begin{bmatrix}
6 & 24 \\
1 & 7
\end{bmatrix} \]

### Prepare the Plaintext

For plaintext "HELLO WORLD":

1. Convert to uppercase and remove spaces: "HELLOWORLD"
2. Split into blocks of size 2: "HE", "LL", "OW", "OR", "LD"

### Encrypt Each Block

1. Convert each block to numerical vectors:
   - 'H' = 7, 'E' = 4
   - 'L' = 11, 'L' = 11
   - 'O' = 14, 'W' = 22
   - 'O' = 14, 'R' = 17
   - 'L' = 11, 'D' = 3

2. Multiply each vector by the key matrix modulo 26:
   - For block "HE":
     \[
     \begin{bmatrix}
     6 & 24 \\
     1 & 7
     \end{bmatrix}
     \begin{bmatrix}
     7 \\
     4
     \end{bmatrix}
     =
     \begin{bmatrix}
     (6 \times 7 + 24 \times 4) \mod 26 \\
     (1 \times 7 + 7 \times 4) \mod 26
     \end{bmatrix}
     =
     \begin{bmatrix}
     22 \\
     15
     \end{bmatrix}
     \]
   - Convert numerical result to letters: 'W', 'P'

### Decrypt Each Block

1. Compute the inverse of the key matrix modulo 26:
   - For matrix \( K \):
     \[
     K^{-1} = \begin{bmatrix}
     7 & 24 \\
     25 & 6
     \end{bmatrix}
     \]

2. Multiply the inverse matrix by the ciphertext vector modulo 26 to recover plaintext.

## Hill Cipher Rules

1. **Key Matrix:** The matrix used for encryption and decryption must be invertible modulo 26.
2. **Block Size:** Encrypt and decrypt blocks of text, where the block size matches the matrix size.
3. **Matrix Multiplication:** Multiply vectors by matrices and take results modulo 26.

By following these steps and applying the matrix operations, you can successfully encrypt and decrypt messages using the Hill cipher.


In [ ]:
import numpy as np

def hill_cipher(text, key):
    """
    Encrypts or decrypts a text using the Hill cipher.

    Args:
        text: The text to be encrypted or decrypted.
        key: A 3x3 matrix representing the key.

    Returns:
        The encrypted or decrypted text.
    """

    # Convert text to numbers
    text_numbers = [ord(char) - ord('A') for char in text.upper()]

    # Pad the text if necessary
    while len(text_numbers) % 3 != 0:
        text_numbers.append(ord('X') - ord('A'))

    # Group text into 3-tuples
    text_groups = [text_numbers[i:i+3] for i in range(0, len(text_numbers), 3)]

    # Perform matrix multiplication
    result = []
    for group in text_groups:
        result.extend(np.dot(group, key) % 26)

    # Convert numbers back to text
    return ''.join(chr(num + ord('A')) for num in result)

def get_key_matrix():
    """
    Gets a 3x3 key matrix from the user.

    Returns:
        A 3x3 numpy array representing the key matrix.
    """

    key = []
    for i in range(3):
        row = input(f"Enter row {i+1} (space-separated numbers): ").split()
        key.append([int(num) for num in row])
    return np.array(key)

if __name__ == "__main__":
    plaintext = input("Enter the plaintext: ")
    key = get_key_matrix()

    ciphertext = hill_cipher(plaintext, key)
    print("Ciphertext:", ciphertext)

Enter the plaintext: Hello
Enter row 1 (space-separated numbers): 2 3 1
Enter row 2 (space-separated numbers): 5 1 4
Enter row 3 (space-separated numbers): 7 8 6
Ciphertext: HJLTXX


#Decryption

In [ ]:
import numpy as np

def hill_cipher_decrypt(ciphertext, key):

  # Convert ciphertext to numbers
  ciphertext_numbers = [ord(char) - ord('A') for char in ciphertext.upper()]

  # Group ciphertext into 3-tuples
  ciphertext_groups = [ciphertext_numbers[i:i+3] for i in range(0, len(ciphertext_numbers), 3)]

  # Calculate the inverse key
  determinant = np.linalg.det(key)
  if determinant == 0:
    raise ValueError("Singular matrix cannot be used for decryption.")
  inv_key = np.linalg.inv(key) % 26

  # Perform matrix multiplication with the inverse key
  decrypted_numbers = []
  for group in ciphertext_groups:
    decrypted_numbers.extend(np.dot(group, inv_key) % 26)

  # Convert numbers back to text
  decrypted_text = ''.join(chr(int(num) + ord('A')) for num in decrypted_numbers)
  return decrypted_text

def get_key_matrix():
  key = []
  for i in range(3):
    row = input(f"Enter row {i+1} (space-separated numbers): ").split()
    key.append([int(num) for num in row])
  return np.array(key)

if __name__ == "__main__":
  ciphertext = input("Enter the ciphertext: ")
  key = get_key_matrix()

  decrypted_text = hill_cipher_decrypt(ciphertext, key)
  print("Decrypted Text:", decrypted_text)

Enter the ciphertext: HJLTXX
Enter row 1 (space-separated numbers): 2 3 1
Enter row 2 (space-separated numbers): 5 1 4
Enter row 3 (space-separated numbers): 7 8 6
Decrypted Text: TYDRYG


# LAB 5: Diffie-Hellman Key Exchange

1. **Choose Parameters**:
   - Select a large prime number $p$ and a primitive root $g$ (both can be public).

2. **Private Keys**:
   - Alice selects a private key $a$.
   - Bob selects a private key $b$.

3. **Compute Public Keys**:
   - Alice computes her public key $A = g^a \mod p$.
   - Bob computes his public key $B = g^b \mod p$.
   - They exchange their public keys.

4. **Compute Shared Secret**:
   - Alice computes the shared secret: $s = B^a \mod p$.
   - Bob computes the shared secret: $s = A^b \mod p$.

5. **Shared Secret Established**:
   - Both parties now share the same secret $s$ which can be used for encrypted communication.


In [ ]:
import random
def modular_exponentiation(base, exponent, modulus):
    return (base ** exponent) % modulus

def generate_prime():
    num = random.randint(10, 100)
    while not is_prime(num):
        num = random.randint(10, 100)
    return num

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

if __name__ == "__main__":
    p = generate_prime()
    g = 5
    print(f"Prime (p): {p}")
    print(f"Primitive Root (g): {g}")

    a = random.randint(1, p - 1)  # Alice's private key
    A = modular_exponentiation(g, a, p)  # Alice's public key

    b = random.randint(1, p - 1)  # Bob's private key
    B = modular_exponentiation(g, b, p)  # Bob's public key

    print(f"Alice's Public Key (A): {A}")
    print(f"Bob's Public Key (B): {B}")

    shared_secret_alice = modular_exponentiation(B, a, p)
    shared_secret_bob = modular_exponentiation(A, b, p)

    print(f"Alice's Shared Secret: {shared_secret_alice}")
    print(f"Bob's Shared Secret: {shared_secret_bob}")

    # Verify
    if shared_secret_alice == shared_secret_bob:
        print("Shared secret established successfully!")
    else:
        print("Failed to establish shared secret.")


Prime (p): 53
Primitive Root (g): 5
Alice's Public Key (A): 49
Bob's Public Key (B): 50
Alice's Shared Secret: 46
Bob's Shared Secret: 46
Shared secret established successfully!


# LAB 6: RSA Algorithm Technique


1. **Key Generation**:
   - Choose two large prime numbers $p$ and $q$.
   - Compute $n = p \times q$.
   - Calculate the totient $\phi(n) = (p - 1)(q - 1)$.
   - Choose a public exponent $e$ such that $1 < e < \phi(n)$ and $\text{gcd}(e, \phi(n)) = 1$.
   - Compute the private exponent $d$ such that $d \cdot e \equiv 1 \mod \phi(n)$.

2. **Encryption**:
   - Convert the plaintext message into numerical form.
   - Encrypt the message using the public key:
     $$
     \text{ciphertext} = \text{plaintext}^e \mod n
     $$

3. **Decryption**:
   - Decrypt the ciphertext using the private key:
     $$
     \text{plaintext} = \text{ciphertext}^d \mod n
     $$


In [ ]:
import random

# Function to check if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

# Function to generate a random prime number
def generate_prime():
    num = random.randint(10, 100)
    while not is_prime(num):
        num = random.randint(10, 100)
    return num

# Function to calculate gcd
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

# Function to calculate modular inverse
def mod_inverse(e, phi):
    for i in range(1, phi):
        if (e * i) % phi == 1:
            return i
    return None

# Function to generate RSA keys
def generate_keys():
    p = generate_prime()
    q = generate_prime()
    n = p * q
    phi = (p - 1) * (q - 1)

    e = 3
    while gcd(e, phi) != 1:
        e += 2

    d = mod_inverse(e, phi)
    return (e, n), (d, n)

# Function to encrypt plaintext
def encrypt(plain_text, public_key):
    e, n = public_key
    cipher_text = [(ord(char) ** e) % n for char in plain_text]
    return cipher_text

# Function to decrypt ciphertext
def decrypt(cipher_text, private_key):
    d, n = private_key
    plain_text = ''.join([chr((char ** d) % n) for char in cipher_text])
    return plain_text

# Main execution
if __name__ == "__main__":
    public_key, private_key = generate_keys()

    print("Public Key (e, n):", public_key)
    print("Private Key (d, n):", private_key)

    message = input("Enter a message to encrypt: ")
    encrypted_message = encrypt(message, public_key)
    print("Encrypted message:", encrypted_message)

    decrypted_message = decrypt(encrypted_message, private_key)
    print("Decrypted message:", decrypted_message)


Public Key (e, n): (5, 7081)
Private Key (d, n): (2765, 7081)
Enter a message to encrypt: hello
Encrypted message: [4391, 5874, 419, 419, 3742]
Decrypted message: hello


# LAB 7: MD5
---
# Understanding How MD5 Works

## What is MD5?
MD5 (Message Digest Algorithm 5) is a cryptographic hash function that:
1. Takes an input of any size.
2. Produces a fixed 128-bit (32 hexadecimal characters) hash value.
3. Is commonly used for checksums to verify data integrity.

**Key Properties of MD5:**
- **Deterministic:** The same input always gives the same hash.
- **Fixed Output:** Outputs a 128-bit hash, regardless of input size.
- **Fast Computation:** Suitable for quick hashing tasks.
- **Not Secure for Cryptography:** Vulnerable to collisions (two different inputs producing the same hash).

---

## How Does MD5 Work?

### 1. Preprocessing (Message Padding)
- **Purpose:** Ensure the message length is a multiple of 512 bits.
- Steps:
  1. Append a `1` bit to the message.
  2. Add `0` bits until the length is 448 bits modulo 512.
  3. Append the original message length as a 64-bit integer.

### 2. Dividing into Chunks
- The padded message is divided into 512-bit chunks.
- Each chunk is processed sequentially.

### 3. Initializing Buffers
- Four 32-bit registers (`A, B, C, D`) are initialized with predefined constants.

### 4. Processing Each Chunk
- Each chunk is processed with:
  - Logical functions (AND, OR, XOR, NOT).
  - Additions and rotations.
  - Constants and pre-defined shifts.

### 5. Final Hash
- After processing all chunks, the buffers (`A, B, C, D`) are concatenated to produce the final 128-bit (32 hexadecimal characters) hash.

---

## Why Use MD5?
- Suitable for non-cryptographic purposes like checksums and data validation.
- Ensures basic data integrity by detecting accidental modifications.

---


In [ ]:
import hashlib

# Function to compute the MD5 hash of a given message
def md5_hash(message):
    # Create an MD5 hash object
    md5 = hashlib.md5()
    # Update the hash object with the message (encoded to bytes)
    md5.update(message.encode('utf-8'))
    # Return the hexadecimal digest of the hash
    return md5.hexdigest()

# User input for the message to hash
message = input("Enter the message to hash using MD5: ")
# Compute and print the MD5 hash
print("MD5 Hash:", md5_hash(message))

Enter the message to hash using MD5: hello
MD5 Hash: 5d41402abc4b2a76b9719d911017c592


# LAB 8: SHA-1
---
# Understanding How SHA-1 Works

## What is SHA-1?
SHA-1 (Secure Hash Algorithm 1) is a cryptographic hash function that:
1. Takes an input message of arbitrary length.
2. Produces a fixed 160-bit (40 hexadecimal characters) hash value.
3. Ensures integrity by creating a unique "fingerprint" for the input.

**Key Properties of SHA-1:**
- **Deterministic:** The same input always results in the same hash.
- **Fixed Output:** Always produces a 160-bit hash, regardless of input size.
- **Non-reversible:** It's computationally infeasible to reconstruct the original message from the hash.

---

## How Does SHA-1 Work?

### 1. Preprocessing (Message Padding)
- **Purpose:** Ensure the message length is a multiple of 512 bits.
- Steps:
  1. Append a single `1` bit to the message.
  2. Add `0` bits until the message length is 448 bits modulo 512.
  3. Append the original message length (in bits) as a 64-bit big-endian integer.

### 2. Processing in 512-bit Chunks
- The padded message is divided into 512-bit chunks.
- Each chunk is processed to update the internal hash state.

### 3. Message Schedule (Word Expansion)
- Each 512-bit chunk is split into sixteen 32-bit words.
- These words are expanded into 80 words using XOR and bitwise operations.

### 4. Hash Computation (80 Iterations)
- Five initial hash values (`h0, h1, h2, h3, h4`) are defined.
- The algorithm uses four main functions (`f`) and constants (`k`) based on the iteration:
  1. 0–19: Logical `AND` and `NOT` operations.
  2. 20–39: XOR operations.
  3. 40–59: Logical `AND` and `OR` combinations.
  4. 60–79: XOR operations.
- The hash values are updated iteratively using a combination of bitwise operations, additions, and logical functions.

### 5. Final Hash
- After processing all chunks, the five hash values are concatenated to form the final 160-bit (40 hex characters) hash.

---

## Why Use SHA-1?
- Ensures data integrity by verifying that the message has not been altered.
- Commonly used in applications like checksum validation, digital signatures, and certificates.

---

## Steps in the Code
1. The **`pad_message`** function ensures the input message is padded according to SHA-1 rules.
2. The **`process_block`** function processes each 512-bit chunk and updates the hash values.
3. After all chunks are processed, the final hash value is returned as a 40-character hexadecimal string.

---

## Example Execution
1. User inputs a message to hash:
```bash
Enter the message to hash using SHA-1: Hello, World!
```
2. The SHA-1 function computes the hash:
```bash
SHA-1 Hash: d3486ae9136e7856bc42212385ea797094475802
```

This ensures a unique fingerprint for the input message.

In [ ]:
import struct

def sha1(message):
    # Preprocessing: Padding the message
    def pad_message(message):
        original_byte_len = len(message)
        original_bit_len = original_byte_len * 8
        message += b'\x80'  # Append a single '1' bit
        while (len(message) * 8 + 64) % 512 != 0:
            message += b'\x00'  # Append '0' bits

        # Append the original message length as a 64-bit big-endian integer
        message += struct.pack('>Q', original_bit_len)
        return message

    # Process the message in successive 512-bit chunks
    def process_block(chunk, h0, h1, h2, h3, h4):
        # Break chunk into sixteen 32-bit big-endian words w[0..15]
        w = list(struct.unpack('>16I', chunk)) + [0] * 64

        # Extend the sixteen 32-bit words into eighty 32-bit words
        for i in range(16, 80):
            w[i] = ((w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]) << 1) & 0xffffffff

        # Initialize hash value for this chunk
        a, b, c, d, e = h0, h1, h2, h3, h4

        # Main loop (80 iterations)
        for i in range(80):
            if 0 <= i <= 19:
                f = (b & c) | (~b & d)
                k = 0x5A827999
            elif 20 <= i <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= i <= 59:
                f = (b & c) | (b & d) | (c & d)
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = (a << 5 & 0xffffffff) + f + e + k + w[i]
            e = d
            d = c
            c = (b << 30) & 0xffffffff
            b = a
            a = temp & 0xffffffff

        # Add this chunk's hash to result so far
        h0 = (h0 + a) & 0xffffffff
        h1 = (h1 + b) & 0xffffffff
        h2 = (h2 + c) & 0xffffffff
        h3 = (h3 + d) & 0xffffffff
        h4 = (h4 + e) & 0xffffffff

        return h0, h1, h2, h3, h4

    # Initial hash values
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0

    # Pad the message
    message = pad_message(message)

    # Process each 512-bit chunk
    for i in range(0, len(message), 64):
        chunk = message[i:i + 64]
        h0, h1, h2, h3, h4 = process_block(chunk, h0, h1, h2, h3, h4)

    # Produce the final hash value (40 hexadecimal characters)
    final_hash = ''.join(f'{x:08x}' for x in (h0, h1, h2, h3, h4))
    return final_hash

# User input for the message to hash
user_input = input("Enter the message to hash using SHA-1: ").encode('utf-8')

# Call the SHA-1 function and print the resulting hash
print("SHA-1 Hash:", sha1(user_input))

Enter the message to hash using SHA-1: hello
SHA-1 Hash: 3ceaa4b9801c708118badcfe90325476c3d2e1f0


In [ ]:
import hashlib

# Get user input and encode it to bytes
user_input = input("Enter the message to hash using SHA-1: ").encode()

# Create SHA-1 hash object and hash the input message
sha1_hash = hashlib.sha1(user_input).hexdigest()

# Print the resulting hash
print("SHA-1 Hash:", sha1_hash)

Enter the message to hash using SHA-1: Keshwam
SHA-1 Hash: d0fa15a1f0bad31bf6e76c07fa0b503f4ef18cee


# LAB 9: RSA with DSA

---
# Understanding How RSA Digital Signatures Work

## What is a Digital Signature?
A digital signature is a cryptographic technique used to ensure:
1. **Authenticity:** Confirms that the message came from the claimed sender.
2. **Integrity:** Ensures that the message has not been tampered with.
3. **Non-repudiation:** Prevents the sender from denying the message later.

## How Does RSA Digital Signature Work?

### 1. Key Pair Generation
- RSA generates two keys:
  - **Private Key:** Used for signing the message (kept secret).
  - **Public Key:** Used for verifying the signature (shared publicly).

### 2. Signing a Message
- The sender uses their **private key** to create a signature for the message:
  1. The message is hashed using a secure hashing algorithm (e.g., SHA-256).
  2. The hash is encrypted using the sender's private key, creating the signature.

### 3. Verifying the Signature
- The receiver verifies the signature using the sender’s **public key**:
  1. The signature is decrypted using the sender's public key to retrieve the hash.
  2. The receiver hashes the original message.
  3. If the two hashes match:
     - The signature is valid.
     - The message is authentic and unaltered.
  4. If they don’t match:
     - The signature is invalid (message may be tampered or not from the claimed sender).

## Why Use Digital Signatures?
- Ensures trust in communication.
- Protects sensitive data from tampering.
- Widely used in secure communications like emails, software distribution, and contracts.

## Key Steps in the Script
1. Generate an RSA key pair (private and public keys).
2. Use the private key to sign a predefined message.
3. Verify the signature using the public key.

This ensures that the message is from the original sender and has not been modified.



In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature


def generate_keys():
    """
    Generate a new RSA private and public key pair.

    Returns:
        tuple: private_key, public_key
    """
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return private_key, public_key


def sign_message(private_key, message):
    """
    Sign a message using the private key.

    Args:
        private_key: The RSA private key.
        message (bytes): The message to be signed.

    Returns:
        bytes: The signature.
    """
    return private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )


def verify_signature(public_key, message, signature):
    """
    Verify the signature of a message using the public key.

    Args:
        public_key: The RSA public key.
        message (bytes): The original message.
        signature (bytes): The signature to verify.

    Returns:
        None
    """
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("Signature is valid.")
    except InvalidSignature:
        print("Signature is invalid.")


if __name__ == "__main__":
    # Generate RSA key pair
    private_key, public_key = generate_keys()

    # Message to be signed
    message = b"Secure message"

    # Sign the message
    signature = sign_message(private_key, message)
    print("Signature:", signature)

    # Verify the signature
    verify_signature(public_key, message, signature)


Signature: b"\xb5\t\xd7\xe0\x81'\x86_\xf8m\xcc`\xe5`\x072\x9e\x8at\x83.\xa45N}\x1al|cD\xa0N<\xc9\x11Or,\xc1^G\xe412\xacS'\x0c\x0bb\xea\x139\x80\xe6M\x9d\x1a\xc84\xd3\xf4`\x01\xaaA\x10rJ;\x16\xbf\xed\x8b\x937Z|*\xa1s \xa0\x08\xe8\x1c\xd2\xd5\xa4\xb2\x973\xc4\x15 \xd9\xf2\xa8\x1d\xddc\xd2U\xf7f\xc2\xfcj9M\x91\x97\xdcA\xec\xef\xb6L\xa8\x93\xec4\x98\xee\xe6\x9e\xbda\x02\x85\xb6\x0b\xe5\x9f_;:\x82`\xe5\x15\x8e\xbe\xa4\x15GO\xa6\xac\x8aI\xe8:\xf3\xca\x06\xda\x0fdg2F\xac\x90y\x99x\xcaF\xcf\x82\x86\xdc\xb8LTO\x02\xfe;\x1f\xc6\x1eJ\x81,\xf6\xfah2\xe6}.\x1b\xeb\x8e\xf9<\xb9Xep\r\rT\xfd\x9e\xa3%\x93\x03O\xc7\xf6'\xe6\x97\xa7\x176\xc5\xac\xf5;\x06\xaf\x8e\x82j\xa9\xa3 \x1bW9\x16%~\xc7\xe5\xc0g\xc4\xa6\x18nT\xb4\xef\xb2\x00\xa9\x90'L_"
Signature is valid.


# LAB 10: DSA

---

# Understanding How DSA Digital Signatures Work

## What is a Digital Signature?
A digital signature is a cryptographic technique used to:
1. **Authenticate** the sender of a message.
2. **Ensure Integrity** of the message, confirming it hasn’t been tampered with.
3. **Provide Non-repudiation,** so the sender cannot deny signing the message.

## How Does DSA Digital Signature Work?

### 1. Key Pair Generation
- **DSA (Digital Signature Algorithm)** generates:
  - **Private Key:** Used by the sender to sign the message (kept secret).
  - **Public Key:** Used by the receiver to verify the signature (shared publicly).

### 2. Signing a Message
- The sender uses their **private key** to sign the message:
  1. The message is hashed using a secure hashing algorithm (e.g., SHA-256).
  2. DSA uses the private key and the hash to create a signature (mathematical operations ensure its uniqueness for this message).

### 3. Verifying the Signature
- The receiver verifies the signature using the sender’s **public key**:
  1. The signature is checked against the message hash using the public key.
  2. If the verification succeeds:
     - The signature is valid.
     - The message is authentic and unaltered.
  3. If the verification fails:
     - The signature is invalid (message may be tampered or not from the claimed sender).

## Why Use DSA for Digital Signatures?
- Provides a lightweight and efficient signature process.
- Ensures secure communication and message integrity.
- Commonly used in secure protocols like TLS and SSH.

## Key Steps in the Script
1. Generate a DSA key pair (private and public keys).
2. Use the private key to sign a message.
3. Verify the signature using the public key.

This guarantees that the message is authentic, originated from the sender, and remains unaltered during transmission.


In [13]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes
from cryptography.exceptions import InvalidSignature


def generate_dsa_keys():
    """
    Generate a new DSA private and public key pair.

    Returns:
        tuple: private_key, public_key
    """
    private_key = dsa.generate_private_key(key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key


def sign_message(private_key, message):
    """
    Sign a message using the DSA private key.

    Args:
        private_key: The DSA private key.
        message (bytes): The message to be signed.

    Returns:
        bytes: The generated signature.
    """
    return private_key.sign(
        message,
        hashes.SHA256()
    )


def verify_signature(public_key, message, signature):
    """
    Verify a signature using the DSA public key.

    Args:
        public_key: The DSA public key.
        message (bytes): The original message.
        signature (bytes): The signature to verify.

    Returns:
        None
    """
    try:
        public_key.verify(
            signature,
            message,
            hashes.SHA256()
        )
        print("Signature is valid.")
    except InvalidSignature:
        print("Signature is invalid.")


if __name__ == "__main__":
    # Generate DSA keys
    private_key, public_key = generate_dsa_keys()

    # Message to be signed
    message = b"Secure message using DSA"

    # Sign the message
    signature = sign_message(private_key, message)
    print("Signature:", signature)

    # Verify the signature
    verify_signature(public_key, message, signature)


Signature: b'0E\x02 u\xfc\xbc\xe5Y\xac\xd5fdF\x7f\xb1\x99\xee\xaa\x1f\x12\x87\x1f\x8fWg\xd8\xa5\xcd\x9ejh`\xbeg\x81\x02!\x00\x93\xa0Dqy\xddc\xa9x\xf44\xf1\xeag\xab\x92\xf9r5*K\xbb\x0f\xb8\xbf\xe7\xd1\x137b\x0cY'
Signature is valid.
